# LLM agent Hackathon 
## Alireza Ghafarollahi, MIT

# AG2: Building LLM-powered multi-agent systems

In [ ]:
!pip install ag2
!pip install openai

In [1]:
import json
import os

with open('config.json') as f:
    config = json.load(f)

OPENAI_API_KEY = config["api_key"]
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Example: Group chat for an online shopping problem

## Let's add a tool that retreives the laptop price

- ## Product Finder

- ## Purchase Advisor

- ## Checkout Assistant

- ## Customer (human)

In [7]:
from typing import Annotated

def laptop_price(model: Annotated[str, "Laptop model"]):
    """Retrieve the price of the laptop model in $."""

    import random

    price = random.randint(1000, 3000)
    
    return f'The price of laptop model {model}: {price}$'

In [8]:
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import AutoPattern

llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")

# Product finder agent
finder_message = """You are a smart product finder agent.
Given a general need or interest, suggest a suitable product someone might want to buy online. Mention the product name and a one-line reason why it's a good fit.
"""

# Purchase advisor agent
advisor_message = """You are a purchase advisor agent.
Given a product name, generate pros and cons, price estimate, and suggest one alternative product with reasoning.
"""

price_message = """You are an assistant agent.
Given a chosen product, provide the exact price of the laptop model.
"""

# Checkout assistant agent
checkout_message = """You are a checkout assistant agent.
Given a chosen product, provide a brief checkout summary including product, estimated price, shipping method, and a friendly thank-you message.
"""

# Customer initiates the shopping process
customer_message = """You are an online shopper.
You describe your needs, review purchase advice (with one round of follow-up), and then confirm a product to buy.
"""

with llm_config:
    product_finder = ConversableAgent(
        name="finder_agent",
        system_message=finder_message,
        description="Call this agent if you want to find new products."
    )

    purchase_advisor = ConversableAgent(
        name="advisor_agent",
        system_message=advisor_message,
        description="Call this agent if you need advice or seek more information about a product."
    )

    checkout_assistant = ConversableAgent(
        name="checkout_agent",
        system_message=checkout_message,
        description="Call this agent for a brief checkout summary including product, price, shipping methods."
    )

    price_assistant = ConversableAgent(
        name="price_agent",
        system_message=price_message,
        description="Call this agent to get the exact price of the laptop.",
        functions=[laptop_price]
    )

customer = ConversableAgent(
    name="customer_agent",
    system_message=customer_message,
    llm_config=False,
    human_input_mode='ALWAYS'
)

In [9]:
pattern = AutoPattern(
    initial_agent=product_finder,         # Start with the customer
    agents=[product_finder, purchase_advisor, checkout_assistant, price_assistant],           # All agents in the group chat
    user_agent=customer,                            # Provide our human-in-the-loop agent
    group_manager_args={"llm_config": llm_config}  # Config for group manager
)

In [10]:
initial_prompt = """I'm looking for a Laptop suitable for training deep learning models."""

result, context_variables, last_agent = initiate_group_chat(
    pattern=pattern,
    messages=initial_prompt,                     # Initial request with transactions
)

customer_agent (to chat_manager):

I'm looking for a Laptop suitable for training deep learning models.

--------------------------------------------------------------------------------

Next speaker: finder_agent


>>>>>>>> USING AUTO REPLY...
finder_agent (to chat_manager):

Consider the **ASUS ROG Zephyrus G14**. It features a powerful AMD Ryzen 9 processor and NVIDIA GeForce RTX 3060 graphics, making it ideal for handling intensive deep learning tasks efficiently.

--------------------------------------------------------------------------------

Next speaker: advisor_agent


>>>>>>>> USING AUTO REPLY...
advisor_agent (to chat_manager):

**ASUS ROG Zephyrus G14**

**Pros:**
1. **Powerful Performance:** Equipped with an AMD Ryzen 9 processor and NVIDIA GeForce RTX 3060 GPU, it's capable of handling demanding deep learning tasks with ease.
2. **Portability:** Lightweight and compact design, making it easy to carry around for on-the-go training or development.
3. **High Refresh Rate Di

Replying as customer_agent. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Can you also suggest a Dell laptop?


customer_agent (to chat_manager):

Can you also suggest a Dell laptop?

--------------------------------------------------------------------------------

Next speaker: checkout_agent


>>>>>>>> USING AUTO REPLY...
checkout_agent (to chat_manager):

Sure! I recommend the **Dell XPS 15**. 

**Dell XPS 15 Features:**
- **Powerful Performance:** It comes equipped with options for Intel Core i7 or i9 processors and NVIDIA GeForce GTX 1650 Ti or RTX 3050, ensuring great performance for deep learning tasks.
- **Stunning Display:** The 15.6-inch 4K display delivers vibrant visuals and more screen real estate for coding and data visualization.
- **Sleek Design:** With its premium build quality and professional aesthetics, it’s perfect for both work and casual use.

**Price Estimate:** The Dell XPS 15 usually ranges around **$1,300 - $2,000**, depending on the specifications.

Let me know if you’d like to proceed with this option or need more information!

---------------------------------------

Replying as customer_agent. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit



>>>>>>>> TERMINATING RUN (dd41ebd4-0dab-4546-b857-5b40393538f6): User requested to end the conversation

>>>>>>>> TERMINATING RUN (5533e8ec-27ab-4b0d-8fc7-c7283058e8e5): No reply generated
